In [2]:
# Permet de tout executer au lancement du notebook + conserver le notebook actif pendant 2h
from IPython.display import Javascript
from masquer import *
Javascript("""
function repeter(){
IPython.notebook.kernel.execute("a=1");
}
// execute a = 1 en python toutes les 8 minutes pendant 2h
let timerId = setInterval(() => repeter(), 4800);
setTimeout(() => { clearInterval(timerId); alert('fin de cession'); }, 7200000);

// Supprimer la taille limite pour la sortie d'une cellule
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
};
IPython.notebook.kernel.execute("url = '" + window.location + "'");

// Exécuter toutes les cellule du notebook
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            
                
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
                Jupyter.actions.call('jupyter-notebook:hide-header')

        }
    );""")

<IPython.core.display.Javascript object>

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
HTML("""<style>
h1 {
  font-family: 'Permanent Marker', cursive;
  text-align: center;
  color: red;
  
}
ol {
  list-style-position: inside;
  margin-left: 1em;
  list-style-position: outside;
}
h2 {
  font-family: 'Permanent Marker', cursive;
  color: blue;
}
h3 {
  font-family: 'Permanent Marker', cursive;

}
</style>""")

# <span style="color:red;"><center>CHAPITRE 2 - Recursivité</center></span>

## III. Limites

 ### III.1. Limite de la taille de la pile d'appel


 En python par défaut, on ne peut pas réaliser plus de 1000 appels imbriqués sinon on se retrouve avec l'erreur
```
RecursionError: maximum recursion depth exceeded in comparison
```
Sous jupyter, cette limite est portée à 3000 par défaut (ce qui permet en réalité d'aller un tout petit peu en dessous).


Ce système est là pour éviter les problèmes de boucle infinie qui peuvent se poser très facilement quand on écrit une fonction récursive et que l'on oublie la condition d'arrêt.

 <img src="https://preview.redd.it/0wap3cp4khm01.jpg?auto=webp&s=3dec95610a58f519cdc503061ef41bf85ed63295" width="500" height="500">

La solution est d'utiliser la fonction [**setrecursionlimit**](https://docs.python.org/3/library/sys.html#sys.setrecursionlimit) du module **sys**
Dans la documentation python, on peut lire:
>Set the maximum depth of the Python interpreter stack to limit. This limit prevents infinite recursion from causing an overflow of the C stack and crashing Python.

>The highest possible limit is platform-dependent. A user may need to set the limit higher when they have a program that requires deep recursion and a platform that supports a higher limit. This should be done with care, because a too-high limit can lead to a crash.


On reprend la suite $u_n$ rappelée ci-dessous.

In [4]:
def u(n):
    if n == 0:
        return 2
    else:
        x = u(n-1)
        return 0.5*(x + 3 / x)

### Exercice 1.
1. Calculez u(10000)
2. Proposez un fonction **u3** sur le modèle de **u(n)** qui prend pour argument un entier **a** et un float **epsilon** (plus ce qui vous sera nécessaire) et qui renvoie $u_n \approx \sqrt{a}$ lorsque $|u_n - u_{n-1}| < epsilon$. Votre fonction devra aussi renvoyer le nombre d'appels imbriqués réalisés.

In [5]:
# Question 1.

import sys
sys.setrecursionlimit(15000)
u(10000)

1.7320508075688772

In [6]:
# Question 2.

def u3(a, epsilon, u = 2, n = 0):
    assert type(a) == int
    assert type(epsilon) == int or type(epsilon) == float
    
    n += 1
    current = 0.5*(u + a / u) # Ce calcul est la fonction racine carré.
    
    if (abs(current - u) < epsilon):
        return current, n
    else :
        return u3(a, epsilon, current, n)
    
u3(3, 1e-10)

(1.7320508075688772, 5)

In [13]:
# Pour coder un float en binaire il faut multiplier par deux et prendre l'entier qui reste
# j'usqu'à qu'il n'y ait plus de décimale :
    
#    0.1875 * 2 --> 0.375 * 2 --> 0.75 * 2 --> 1.5 
#                                          --> 0.5 * 2 --> 1
#                                          
#    0 0 0 1 1
    
#    Cela va faire une boucle infini pour 0.1
    
#    0.1 * 2 --> 0.2 * 2 --> 0.4 * 2 --> 0.8 * 2 --> 1.6
#                                                --> 0.6 * 2 --> 1.2
#                                                            --> 0.2 * 2 --> 0.4 ... On revient au début
                                                            
#    0 0 0 0 1 1 0 0 ect

### III.2. Limite de complexité

La fonction **u(n)** précédente appelle une seule fois la fonction **u(n-1)** et ainsi de suite. Le résultat est un appel pour chaque n, avec chaque appel réalisé en temps constant d'où une complexité en O(n) (en gros un temps d'exécution proportionnel à n sur une même machine) comme dans le cas de l'utilisation d'un boucle for pour le calcul.


Si on utilise la fonction **u(n)** ci-dessous, u(n) appelle deux fois u(n-1) qui appellent chacune deux fois u(n-2) etc... On se retrouve avec $2^n$ appels et donc un temps d'exécution exponentiel! (le temps est proportionnel à $2^n$ et double donc lorsque n augmente de 1. Faison l'expérience:

In [8]:
def u(n):
    if n == 0:
        return 2
    else:
        return 0.5*(u(n-1) + 3 / u(n-1))

from time import time
for i in range(25):
    t = time()
    racinedetrois = u(i)
    print(i,time()-t)

0 1.1920928955078125e-06
1 2.1457672119140625e-06
2 2.384185791015625e-06
3 2.6226043701171875e-06
4 3.814697265625e-06
5 6.198883056640625e-06
6 1.2159347534179688e-05
7 2.4557113647460938e-05
8 4.792213439941406e-05
9 9.512901306152344e-05
10 0.0001976490020751953
11 0.00038051605224609375
12 0.0010716915130615234
13 0.0018777847290039062
14 0.0032231807708740234
15 0.006491661071777344
16 0.014111757278442383
17 0.024845123291015625
18 0.05796313285827637
19 0.12603545188903809
20 0.23517775535583496
21 0.39950013160705566
22 0.8168284893035889
23 1.5974416732788086
24 3.1821627616882324


De manière générale, une fonction récursive qui s'appelle plus d'une fois donnera une complexité exponentielle, il faudra donc bien y faire attention lors du codage d'un fonction récursive.

### Exercice 2. 
La suite de fibonaci est définie par:

$F_0 = 1$

$F_1 = 1$

$F_n = F_{n-1} + F_{n-2}$ $\forall n \in \mathbb{N}, n > 1$
1. Programmez une foncton qui prend comme argument un entier **n** et qui renvoie la valeur de $F_n$.
2. Quelle est l'ordre de grandeur de la valeur de n la plus grande accessible avec cette fonction ?
3. Programmez cette fonction de manière iterative.
4. Quelle est l'ordre de grandeur de la valeur de n la plus grande accessible avec cette fonction ?


In [12]:
def suiteF(n) :
    assert n >= 0
    assert type(n) == int
    
    if n < 2:
        return 1
    else:
        return suiteF(n-1) + suiteF(n-2)
    
suiteF(39)

102334155

In [14]:
# F(39) prends environ 15 seconde
# Le tps augmente par * 1.6 pour chaque incrément de n

# Pour F(100) (en années) ? 100 - 39 = 61
15 * 1.6**61 / (365.25 * 24 * 60**2)

1343712.1186237663

In [ ]:
def F2(n) :
    assert type(n) == int
    assert n >= 0
    
    if n < 2:
        return 1
    
    compteur = 1
    prev1 = 1 # n - 1
    prev2 = 1 # n - 2
    
    while compteur < n:
        compteur += 1
        prev1, prev2 = prev1 + prev2, prev1
        
    return a

# La complexité est de O(n)

Il est possible de remédier à ce problème de complexité de la fonction récursive en utilisant **la mise en cache** ou **mémoïsation**. Il s'agit de stocker dans un tableau la valeur de chaque terme déjà calculé afin d'y accéder rapidement sans le recalculer. Le tableau doit être accessible à tous les niveaux de la pile d'appel.

In [10]:
import sys
sys.setrecursionlimit(10000)

memoire = [1,1]
# Une variable global pose bcp de problèmes, des intéractions entre fonctions non voulues affecteront ton programme par exemple
def F(n):
    global memoire
    if n < len(memoire):
        return memoire[n] 
    else:
        memoire.append(F(n-1) + F(n-2)) # Ne recalcule pas tout comparer à la première fonction
        return memoire[n]

t = time()
F(5000)
print(time()-t)

6276302800488957086035253108349684055478528702736457439025824448927937256811663264475883711527806250329984690249846819800648580083040107584710332687596562185073640422286799239932615797105974710857095487342820351307477141875012176874307156016229965832589137779724973854362777629878229505500260477136108363709090010421536915488632339240756987974122598603591920306874926755600361865354330444681915154695741851960071089944015319300128574107662757054790648152751366475529121877212785489665101733755898580317984402963873738187000120737824193162011399200547424034440836239726275765901190914513013217132050988064832024783370583789324109052449717186857327239783000020791777804503930439875068662687670678802914269784817022567088069496231111407908953313902398529655056082228598715882365779469902465675715699187225655878240668599547496218159297881601061923195562143932693324644219266564617042934227893371179832389642895285401263875342640468017378925921483580111278055044254198382265567395946431803304304326865077

0.013684272766113281


Et l'on peut reprendre les calculs là ou on s'était arrêté car tout est resté en mémoire

In [11]:
t = time()
F(5000)
print(time()-t)

#  Le tableau existe déjà donc sera encore plus rapide

6276302800488957086035253108349684055478528702736457439025824448927937256811663264475883711527806250329984690249846819800648580083040107584710332687596562185073640422286799239932615797105974710857095487342820351307477141875012176874307156016229965832589137779724973854362777629878229505500260477136108363709090010421536915488632339240756987974122598603591920306874926755600361865354330444681915154695741851960071089944015319300128574107662757054790648152751366475529121877212785489665101733755898580317984402963873738187000120737824193162011399200547424034440836239726275765901190914513013217132050988064832024783370583789324109052449717186857327239783000020791777804503930439875068662687670678802914269784817022567088069496231111407908953313902398529655056082228598715882365779469902465675715699187225655878240668599547496218159297881601061923195562143932693324644219266564617042934227893371179832389642895285401263875342640468017378925921483580111278055044254198382265567395946431803304304326865077

0.008120536804199219


<a href="TD_02_2.ipynb">TD_02_2</a>
    
<center> <a href="cours_02_2.ipynb"><= PRECEDENT</a></center>